## This notebook is for creating and testing different models on classifying a salary range for a job posting

In [161]:
import pandas as pd
import numpy as np
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer

In [162]:
df = pd.read_csv(os.path.dirname(os.getcwd())+"\\EDA\\clean_data.csv")
df.head()

,Unnamed: 0,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Type of ownership,Industry,Sector,Revenue,new_title,min_salary,max_salary,loc_state
0,0,"Data Analyst, Center on Immigration and Justic...",$37K-$66K,Are you eager to roll up your sleeves and harn...,3.2,Vera Institute of Justice,"New York, NY","New York, NY",201 to 500 employees,Nonprofit Organization,Social Assistance,Non-Profit,$100 to $500 million (USD),Data Analyst,37.0,66.0,NY
1,1,Quality Data Analyst,$37K-$66K,Overview\n\nProvides analytical and technical ...,3.8,Visiting Nurse Service of New York,"New York, NY","New York, NY",10000+ employees,Nonprofit Organization,Health Care Services & Hospitals,Health Care,$2 to $5 billion (USD),Data Analyst,37.0,66.0,NY
2,2,"Senior Data Analyst, Insights & Analytics Team...",$37K-$66K,We’re looking for a Senior Data Analyst who ha...,3.4,Squarespace,"New York, NY","New York, NY",1001 to 5000 employees,Company - Private,Internet,Information Technology,Unknown / Non-Applicable,Analytics Solutions,37.0,66.0,NY
3,3,Data Analyst,$37K-$66K,Requisition NumberRR-0001939\nRemote:Yes\nWe c...,4.1,Celerity,"New York, NY","McLean, VA",201 to 500 employees,Subsidiary or Business Segment,IT Services,Information Technology,$50 to $100 million (USD),Data Analyst,37.0,66.0,NY
4,5,Data Analyst,$37K-$66K,About Cubist\nCubist Systematic Strategies is ...,3.9,Point72,"New York, NY","Stamford, CT",1001 to 5000 employees,Company - Private,Investment Banking & Asset Management,Finance,Unknown / Non-Applicable,Data Analyst,37.0,66.0,NY


In [163]:
cols_to_keep = ["Rating",'Size','Type of ownership','Industry','Sector','Revenue','new_title','loc_state']
# for now, drop all na values
df = df.dropna(axis=0)
X = df[cols_to_keep]
y = df["Salary Estimate"]

In [159]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) # Train + val & test

numeric_features = X.select_dtypes(include=np.number).columns.tolist()
categorical_features = [c for c in X.columns if c not in numeric_features]
cat_idx = [X.columns.get_loc(col) for col in categorical_features]


columnTransformer = ColumnTransformer(
    transformers=[
        ('cat', OrdinalEncoder(), categorical_features)])

X_train = columnTransformer.fit_transform(X_train)
X_test = columnTransformer.fit_transform(X_test)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=42) # Train & val

